In [11]:
'''
This cell is only to prepare the example
'''
import os, json
os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from ESObservation import Observation, ResultValue, ES, LocationValue, DatationValue
from datetime import datetime
from pprint import pprint
import requests as rq

In [12]:
''' 
Case 1: Simple sensor

A sensor encode and send an Observation.
The server decode the Observation and store it into a database.
'''
# simulation of a measure
time = "2021-05-05T10:08"
coord = [2.35, 48.87]
prop = {"prp":"Temp"}
res = 25.0

# Observation creation and encoding to Json or to binary data in the sensor
ob_sensor = Observation((time, coord, prop, res))
payload1 = ob_sensor.json           # if the payload is character payload
print(len(payload1), payload1)
payload2 = ob_sensor.to_bytes()     # if the payload is binary payload
print(len(payload2), payload2)
# next : send the payload with the network protocol

# data decoding in the server
ob_receive1 = Observation()
ob_receive1.from_json(payload1)
ob_receive2 = Observation()
ob_receive2.from_bytes(payload2)
print(ob_receive2.json == ob_receive1.json == ob_sensor.json)   # it's True !!

# and store it in the database (example with NoSQL DataBase)
jsonStore = ob_receive1.to_json(storage=True)    # add 'information' in the json to facilitate the research in the database
pprint(json.loads(jsonStore))
url = "https://webhooks.mongodb-realm.com/api/client/v2.0/app/observation_app-wsjge/service/postObs/incoming_webhook/api?secret=10minutes"
r = rq.post(url, data=jsonStore)  
print("réponse : ", r.status_code, "\n")


137 {"type":"observation","datvalue":"2021-05-05T10:08:00","locvalue":[2.35, 48.87],"prpvalue":{"prp": "Temp", "unit": "°C"},"resvalue":25.0}
32 b'\xa0U\xe5\x07\x05\x05\n\x08\x004\xe0\x94f\x01`\xf8 \x1dv\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x90@N'
True
{'datvalue': '2021-05-05T10:08:00',
 'information': {'axes': [],
                 'boudingBox': [2.35, 48.87, 2.35, 48.87],
                 'complet': True,
                 'dim': 0,
                 'measureRate': 1.0,
                 'nvaldat': 1,
                 'nvalloc': 1,
                 'nvalprop': 1,
                 'nvalres': 1,
                 'score': 111,
                 'timeBox': ['2021-05-05T10:08:00', '2021-05-05T10:08:00'],
                 'typedat': 'datationValue',
                 'typeloc': 'locationValue',
                 'typeobs': 'obs',
                 'typeprop': 'propertyValue',
                 'typeres': 'resultValue'},
 'locvalue': [2.35, 48.87],
 'prpvalue': {'prp': 'Temp', 'unit': '°C'},
 'r

In [13]:
'''
case 2 : minimize data in operation

In this example, the sensor is stationary and the property is fixed.
In the initialization phase, an Observation with only Property and Location is sent.
In the operation phase, an Observation with only Result is sent.
The complete observation is reconstructed in the server.
'''

# initialization phase (sensor or server) -> once
coord = [2.35, 48.87]
prop = {"prp":"Temp"}
ob_init = Observation(location=coord, property=prop)
print(ob_init.json)

# operation phase (sensor) -> regularly
res = 25.0
ob_operat = Observation(result=res)
payload1 = ob_operat.json           # if the payload is character payload
print(len(payload1), payload1)
payload2 = ob_operat.to_bytes()     # if the payload is binary payload
print(len(payload2), payload2)      # the payload is only 4 bytes long !!

# next : send the payload with the network protocol

# data decoding in the server
ob_receive1 = Observation()
ob_receive1.from_json(payload1)
ob_receive2 = Observation()
ob_receive2.from_bytes(payload2)
print(ob_receive1.json == ob_receive2.json == ob_operat.json)   # it's True !!

# complete observation
ob_receive2.addValue(DatationValue(datetime.now()))
ob_receive2.extend(ob_init)
print(ob_receive2.json)


{"type":"observation","locvalue":[2.35, 48.87],"prpvalue":{"prp": "Temp", "unit": "°C"}}
38 {"type":"observation","resvalue":25.0}
4 b'\xa0\x90@N'
True
{"type":"observation","resvalue":25.0,"datvalue":"2022-01-18T12:11:58.553587","locvalue":[2.35, 48.87],"prpvalue":{"prp": "Temp", "unit": "°C"}}
